# 1D Maze Place Cell Analysis

Workflow for analyzing place cells in a 3D maze with arm corridors.
Arm traversals are serialized onto a concatenated 1D axis and analyzed
using the same metrics (SI, stability, place fields) as the 2D pipeline.

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
from tqdm.auto import tqdm

from placecell.dataset import BasePlaceCellDataset
from placecell.visualization import (
    plot_diagnostics,
    plot_graph_overlay,
    plot_occupancy_preview_1d,
    plot_shuffle_test_1d,
    plot_summary_scatter,
)

## Configuration

Point to a maze analysis config and a per-session data paths YAML.

In [ ]:
CONFIG_ID = "pcell_maze_config"
DATA_PATH = Path(
    "/Volumes/ProcData/minizero_analysis/202512round/"
    "202512_analysis_3dmaze/20251219/WL25/WL25_20251219.yaml"
)

ds = BasePlaceCellDataset.from_yaml(CONFIG_ID, DATA_PATH)
print(type(ds).__name__)  # Should print 'MazeDataset'

## Step 1: Load & Preprocess

In [ ]:
ds.load()

In [ ]:
ds.preprocess_behavior()

In [ ]:
if ds.graph_polylines is not None:
    plot_graph_overlay(
        ds.graph_polylines,
        ds.graph_mm_per_pixel,
        arm_order=ds.maze_cfg.arm_order,
        video_frame=ds.behavior_video_frame,
    )
    plt.show()
else:
    print("No behavior graph — skipping graph overlay.")

In [ ]:
ds.deconvolve(progress_bar=tqdm)

## Step 2: Match Events & Occupancy

In [ ]:
ds.match_events()
ds.compute_occupancy()

In [ ]:
plot_occupancy_preview_1d(
    ds.trajectory_1d_filtered,
    ds.occupancy_time,
    ds.valid_mask,
    ds.edges_1d,
    trajectory_1d=ds.trajectory_1d,
    trajectory_1d_all=ds.trajectory_1d_all,
    arm_boundaries=ds.arm_boundaries,
    arm_labels=ds.effective_arm_order,
);

## Step 3: Analyze Units

In [ ]:
ds.analyze_units(progress_bar=tqdm)

In [ ]:
summary = ds.summary()
print(summary)

In [ ]:
plot_summary_scatter(
    ds.unit_results, p_value_threshold=ds.spatial_1d.p_value_threshold,
    n_shuffles=ds.spatial_1d.n_shuffles,
    min_shift_seconds=ds.spatial_1d.min_shift_seconds,
);

In [ ]:
plot_diagnostics(ds.unit_results, p_value_threshold=ds.spatial_1d.p_value_threshold);

## Population Rate Map

Rate maps of all place cells sorted by peak position.

In [ ]:
plot_shuffle_test_1d(
    ds.unit_results,
    ds.edges_1d,
    p_value_threshold=ds.spatial_1d.p_value_threshold,
    arm_boundaries=ds.arm_boundaries,
    arm_labels=ds.effective_arm_order,
);

## Save Bundle

In [ ]:
ds.save_bundle(f"../user_data/bundles/{DATA_PATH.stem}_sandbox.pcellbundle")